### compile-time вычисления

<br />

##### before C++11

Все compile-time вычисления основаны на типах, что больше похоже на метапрограммирование.

Посчитаем факториал во время копиляции:

```c++
template<unsigned N>
struct Factorial
{
    static const int value = N * Factorial<N - 1>::value;
};

template<>
struct Factorial<0>
{
    static const int value = 1;
};

int main()
{
    return Factorial<10>::value + Factorial<3>::value;
}
```

**Вопрос**: сколько будет создано структур `Factorial` компилятором?

<br />

##### C++11

В С++11 появляется слово `cosntexpr` и возможность делать `constexpr`-функции. Однако, функционал ограничен:

* Функция возвращать и принимать может только [literal-типы](https://en.cppreference.com/w/cpp/named_req/LiteralType)
* Функция может иметь только один `return` (и никаких локальных переменных, циклов ...)
* Функция может вызывать только `constexpr`-функции
* Функция может кинуть исключение, чтобы сообщить компилятору, что произошла ошибка.

```c++
constexpr int factorial(int n)
{
    return n < 0 ? throw std::out_of_range("")
                 : n == 0 ? 1
                          : n * factorial(n - 1);
}
```

<br />

Не очень богатый набор возможностей.

Тем не менее, нужно усвоить понятие контекста выполнения `constexpr`-функции: runtime vs compiletime.

```c++
int main(int argc, char **argv)
{
    // значение |argc| неизвестно на этапе компиляции,
    // поэтому компилятор не может посчитать результат
    // заранее, выполнение будет происходить в runtime
    std::cout << factorial(argc);
    
    // значение |const_value| известно на этапе
    // компиляции, поэтому компилятор может вычислить
    // результат заранее и подставить ответ, но, емнип,
    // не обязан (likely compiletime)
    int const_value = 5;
    std::cout << factorial(const_value);
    
    // программист обязывает компилятор вычислить результат
    // на этапе компиляции, если компилятор не справляется,
    // компиляция должна закончится с ошибкой (compiletime)
    constexpr int f1 = factorial(5);
    std::cout << f1;
    
    // ключевое слово volatile отключает оптимизацию,
    // позволяющую компилятору использовать значение
    // |const_value_2| во время компиляции. Результат
    // будет вычиляться в runtime.
    volatile int const_value_2 = 8;
    std::cout << factorial(const_value_2);
    
    // использование результата в compiletime контексте
    // тоже обязывает компилятор произвести вычисления
    // compiletime
    std::array<int, factorial(5)> arr = {};
    static_assert(factorial(2) == 2, "factorial(2) should be equal to 2");
    
    // ошибка компиляции:
    constexpr int f2 = factorial(argc);
}
```

**Замечание**:
* Переменная, объявленная `constexpr`, получает модификатор `const`.
    * Т.о. следующий сценарий невозможен: инициализацию переменной просчитать в compiletime, а в runtime её модифицировать.
    * В С++20 добавили такую возможность.
* Компилятор при вычислении `constexpr`-функций *интерпретирует* код (с кучей проверок), а значит сами вычисления медленные. Насколько и как живут с этим компиляторы - чуть позднее.

<br />

##### constexpr vs undefined behavior

Приятной особенностью `constexpr`-функций является возможность находить Undefined Behavior компилятором:
* Компилятор исполняет код, а кто лучше него знает, сейчас исполняется UB или не UB?
* UB в compiletime-контексте - ошибка компиляции

```c++
constexpr int factorial(int n)
{
    return n < 0 ? throw std::out_of_range("")
                 : n == 0 ? 1
                          : n * factorial(n - 1);
}

int main(int argc, char **argv)
{
    // ошибка компиляции, в процессе вычислений произошло
    // переполнение знакового целого
    constexpr int f2 = factorial(50);
}
```

<br />

Итак, у нас повился план как починить проблему UB в С++:
1. Объявляем все функции (какие сможем) `constexpr`
2. Полностью покрываем функции тестами, которые будут звать функции в compiletime контексте
3. Итого: все сценарии в тестах свободны от UB.

Хорошо, но:
* Тесты должны покрывать сценарии использования.
    * Код `int x = ...; x * 2` при одних сценариях работает, а при других - UB. Тесты должны покрывать именно сценарии использования.
    * Понятно, что функция `int factorial(int x)` на больших `x` будет давать UB. Вопрос в том, есть ли такие сценарии на production.
* Набор функций, которые можно обозвать `constexpr`, не так уж и широк
    * Над этим работает комитет

<br />

##### C++14

Расширен набор `constexpr`-функций. Тело `constexpr`-функции может содержать любые конструкции, кроме:
* `asm`
* `goto`
* Переменных нелитерального типа, `static` и `thread_local`

Уже значительно лучше:
* `asm` и `goto` редко когда нужны (почти никогда)
* зачастую код можно организовать без `static` и `thread_local`
* требование на литеральные типы - очень сильное, `std::vector` уже не вписывается

Примеры:

```c++
constexpr int factorial(int n)
{
    if (n < 0)
        throw std::out_of_range("factorial arg < 0");

    int f = 1;
    for (int i = 2; i <= n; ++i)
        f *= i;
    return f;
}
```

Если немножко подумать, то можно и так:

```c++
template<typename It, typename ValueType>
constexpr std::size_t count(It begin, It end, const ValueType& value)
{
    std::size_t cnt = 0;
    for (; begin != end; ++begin)
        if (*begin == value)
            ++cnt;
    return cnt;
}
```

**Вопрос:**
    
<details>
<summary>Что-нибудь напоминает?</summary>
<p>

Да, стандартные алгоритмы. И их можно объявить `constexpr` и использовать в `constexpr`-функциях.
Этим и занялся комитет, но с [С++20](https://en.cppreference.com/w/cpp/algorithm/count)

</p>
</details>

<br />

##### C++17 if constexpr + constexpr lambda

constexpr lambda:
* Теперь можно навешивать модификатор `constexpr` на лямбда-функции, а можно не навешивать:
    > [doc](https://en.cppreference.com/w/cpp/language/lambda): constexpr: explicitly specifies that the function call operator is a constexpr function. When this specifier is not present, the function call operator will be constexpr anyway, if it happens to satisfy all constexpr function requirements
* Лямбда-функции могут быть literal type

Примеры:

Более естественный вариант:

```c++
constexpr auto get_factorial = [](int i) {
    if (i <= 1)
        return 1;
    return i * get_factorial(i - 1);
};

constexpr int a = get_factorial(5);

int b = get_factorial(argc);
```

Немного чудаковатый вариант с лямбдами из лямбд:

```c++
template <typename I>
constexpr auto adder(I i) {
  return [i](auto j){ return i + j; };
}

constexpr auto add5 = adder(5);

constexpr unsigned v27 = add5(22);
```

**Замечание**: я понятия не имею какое практическое использование можно придумать нововведению с многоуровневыми лямбдами. Поиск по плюсовым блоггерам даёт лишь аналогичные чудаковатые примеры.

<br />

`if constexpr` введён как упрощение для различных механизмов С++, которые иным образом реализовывались через шаблоны или макросы.

Как работает `if constexpr` на демонстрационном примере:

```c++
template <typename T>
auto get_value(T t) {
    if constexpr (std::is_pointer_v<T>)
        return *t;
    else
        return t;
}

// usage
int x = 5;
int* px = &x;
std::cout << get_value(x);
std::cout << get_value(px);
```

При этом:
* `get_value<int>` компилируется в `int get_value(int t) { return t; }`
* `get_value<int*>` компилируется в `int get_value(int*t) { return *t; }`

<br />

Зачем `if constexpr` может быть полезен?

Обычно всё, что реализуется через `if constexpr`, можно реализовать прежними конструкциями языка, но `if constexpr` делает это более читабельным.

**Пример 1**: можно упростить SFINAE (на том же `get_value`).

Вариант реализации `get_value` через SFINAE (вариант через доп. шаблонный параметр):


```c++
template <typename T,
          std::enable_if_t<std::is_pointer<T>{}>* = nullptr>
auto get_value(T t) {
    return *t;
}

template <typename T,
          std::enable_if_t<!std::is_pointer<T>{}>* = nullptr>
auto get_value(T t) {
    return t;
}
```

Против `if constexpr`:

```c++
template <typename T>
auto get_value(T t) {
    if constexpr (std::is_pointer_v<T>)
        return *t;
    else
        return t;
}
```

<br />

**Пример 2**: Упрощение tag dispatching:

Вариант реализации `get_value` через tag dispatching (вариант через тэг как доп. неиспользуемый аргумент переменного типа):

```c++
template <typename T>
auto get_value(T t, std::true_type) {
    return *t;
}

template <typename T>
auto get_value(T t, std::false_type) {
    return t;
}

template <typename T>
auto get_value(T t) {
    return get_value(t, std::is_pointer<T>{}); 
}
```

Против `if constexpr`:

```c++
template <typename T>
auto get_value(T t) {
    if constexpr (std::is_pointer_v<T>)
        return *t;
    else
        return t;
}
```

<br />

**Пример 3:** Упрощение рекурсивных variadic templates:

```c++
sum<5, 3, 2>() == 10;
```

Пример суммы `n` чисел через рекурсивные шаблоны и свёртки
    
```c++
template <int N>
int sum() {
    return N;
}

template <int N1, int N2, int... Ns>
int sum() {
    return N1 + sum<N2, Ns...>();
}
```

Пример через `if constepxr`

```c++
template <int N, int... Ns>
int sum() {
    if constexpr (sizeof...(Ns) == 0)
        return N;
    else
        return N + sum<Ns...>();
}
```

<br />

**Пример 4**: Упрощение structured bindings для классов.

Для структур structured bindings работает из коробки:

```c++
struct Point
{
    float x;
    float y;
};

int main()
{
    std::vector<Point> polyline { ... };
    const auto [x, y] = polyline.front();
    std::cout << x << " " << y;
}
```

Но он не работает для классов, где поля упрятаны в `private`:

```c++
class Person
{
private:
    std::string name;
    std::string surname;
    int age;
    
public:
    ...;
};

const auto [name, surname, age] = make_person(); // compile error
```

Чтобы structured bindings завёлся для `private`-полей, нужно реализовать шаблонный метод `get`.

Вариант реализации через специализацию шаблонов:

```c++
class Person
{
private:
    std::string name;
    std::string surname;
    int age;

public:
    Person() {}
    
    template <std::size_t N> friend auto get(const Person&);
};

namespace std {
    template<> struct tuple_element<0, Person> { using type = std::string; };
    template<> struct tuple_element<1, Person> { using type = std::string; };
    template<> struct tuple_element<2, Person> { using type = int; };
    template<> struct tuple_size<Person>: public integral_constant<size_t, 3> {};
}

template <>
auto get<0>(const Person& p) {
    return p.name;
}

template <>
auto get<1>(const Person& p) {
    return p.surname;
}

template <>
auto get<2>(const Person& p) {
    return p.age;
}


//
// and finally!
// 
const auto& [name, surname, age] = make_person();
```

Вариант реализации через `if constexpr`:

```c++
class Person
{
private:
    std::string name;
    std::string surname;
    int age;

public:
    Person() {}
    
    template <std::size_t N> friend auto get(const Person&);
};

namespace std {
    template<> struct tuple_element<0, Person> { using type = std::string; };
    template<> struct tuple_element<1, Person> { using type = std::string; };
    template<> struct tuple_element<2, Person> { using type = int; };
    template<> struct tuple_size<Person>: public integral_constant<size_t, 3> {};
}

template <std::size_t N>
auto get(const Person& p) {
    if      constexpr (N == 0) return p.name;
    else if constexpr (N == 1) return p.surname;
    else if constexpr (N == 2) return p.age;
    else throw std::out_of_range("Person class supports get<N> only for N in {0, 1, 2}");
}

//
// and finally!
// 
const auto& [name, surname, age] = make_person();
```

<br />

**Пример 5**: замена `#ifdef`.

Здесь, на субъективный взгляд преподавателя, `#ifdef`-ы лучше чем `if constexpr`:
* `#ifdef` они раньше вырезают ненужные ветки кода - на этапе препроцессора. Ветки `if constexpr` должны пройти хотя бы этап лексического анализа.
* `#ifdef`-ы - более краткий вариант

Тем не менее, пример иногда приводят:

```c++
bool is_port_free(int port) {
    #ifdef __linux__
        ...
    #elif __APPLE__
        ...
    #elif __WIN32
        ...
    #endif
}
```

Перевод `#ifdef` через `if constexpr`

```c++
// in some header
enum class OS { Lin, Mac, Win };

//Translate the macros to C++ at a single point in the application
#ifdef __linux__
constexpr OS the_os = OS::Lin;
#elif __APPLE__
constexpr OS the_os = OS::Mac;
#elif __WIN32
constexpr OS the_os = OS::Win;
#endif

// everywhere
bool is_port_free(int port) {
    if constexpr (the_os == OS::Lin) {
        ...
    }
    else if constexpr (the_os == OS::Mac) {
        ...
    }
    else if constexpr (the_os == OS::Win) {
        ...
    }
}
```

Сомнительное улучшение, но, возможно, преподаватель не владеет какими-то хитрыми знаниями.

<br />

##### C++20

Дополнительно в `constexpr`-функциях разрешено:
* иметь try-catch блок
* иметь asm-блок
* пользоваться union
* вызывать виртуальные функции
* dynamic_cast + typeid
* new/delete

В таком коде компилятор теперь должен выдать ошибку, что происходит утечка:
    
```c++
constexpr int foo()
{
    int *x = new int[3];
    x[0] = 0;
    x[1] = 1;
    x[2] = 2;
    return x[0] + x[1] + x[2];
}

int main()
{
    constexpr int res = foo();
    return res;
}
```

**Замечание**: теперь в компилятор встроен инструмент поиска утечек памяти через constexpr!

А в таком коде компилятор найдёт undefined behavior:
    
```c++
constexpr int foo()
{
    int *x = new int[3];
    x[0] = 0;
    x[1] = 1;
    x[2] = 2;
    const int res = x[0] + x[1] + x[2];
    delete x;
    return res;
}

int main()
{
    constexpr int res = foo();
    return res;
}
```

А такой код скомпилируется успешно (когда выйдут компиляторы с поддержкой С++20) - закинуть пример на godbolt на clang (trunk) с опцией -std=c++2a, clang-9 ещё не умеет

```c++
constexpr int foo()
{
    int *x = new int[3];
    x[0] = 0;
    x[1] = 1;
    x[2] = 2;
    const int res = x[0] + x[1] + x[2];
    delete[] x;
    return res;
}

int main()
{
    constexpr int res = foo();
    return res;
}
```

<br />

`constinit` - compiletime инициализация переменных. `constexpr` влечёт за собой `const`, т.е. переменную нельзя менять. Очень редко хочется, чтобы переменную (скорее, глобальную) проинициализировать в compiletime, а потом менять.

```c++
constinit x = factorial(3);

int main()
{
    x += 7;
    return x;
}
```

Пример использования: инициализация счётчиков гистограмм в глобальных переменных, в которые код при выполнении будет накапливать хиты.

```c++
struct HistBucket
{
    std::string_view name;
    std::uint64_t counter;
};

constinit std::array<HistBucket, 3> histograms {
    {"network_requests", 0},
    {"network_failures", 0},
    {"database_reads", 0}
};
```

<br />

`consteval`-функции могут вызываться только в compile-time контексте.

```c++
consteval int square(int i) {
    return i * i;
}

constexpr int x = square(2); // ok
int y = square(argc); // error
```

Пример использования будет ниже.

<br />

`std::is_constant_evaluated()` - проверка, что находимся в compile-time контексте

```c++
constexpr int square(int i) {
    if (std::is_constant_evaluated()) {
        return i * i;  // slow code that allows compile-time
    } else {
        return magic_fast_square(i);  // fast code that disallowed in compile time
    }
}

constexpr int x = square(2); // ok - compiletime, compiletime branch
int y = square(argc); // ok, runtime, runtime branch
```

<br />

##### насколько compiletime медленнее runtime?

Возьмём пример с числами Фибоначи и добавим ничего не делающих вычислений, которые компилятор не может выкинуть в процессе оптимизации:

```c++
#include <iostream>

constexpr std::uint64_t fib(std::uint64_t n)
{
    if (n <= 1)
        return n;

    for (std::uint64_t i = 1; i < 1000; ++i)
        for (std::uint64_t j = 1; j < 1000; ++j)
            if (i * i * i + j * j * j == n * n * n)
                return 0;

    return fib(n - 1) + fib(n - 2);
}

int main(int argc, char** argv)
{
    constexpr std::uint64_t res = fib(10);
    std::cout << res << std::endl;
    return 0;
}
```

Вот что на это говорит g++

```bash
$ g++ --version
g++ (Ubuntu 7.4.0-1ubuntu1~18.04.1) 7.4.0

$ time g++ main_constexpr.cpp

real    0m19,215s
user    0m18,914s
sys     0m0,289s
```

19 секунд компиляции

Что на это скажет clang?

```bash
$ clang++-8 --version
clang version 8.0.0-3~ubuntu18.04.1 (tags/RELEASE_800/final)

$ time clang++-8 main_constexpr.cpp
main_constexpr.cpp:19:29: error: constexpr variable 'res' must be initialized by a constant expression
    constexpr std::uint64_t res = fib(10);
                            ^     ~~~~~~~
main_constexpr.cpp:11:13: note: constexpr evaluation hit maximum step limit; possible infinite loop?
            if (i * i * i + j * j * j == n * n * n)
            ^
main_constexpr.cpp:14:12: note: in call to 'fib(9)'
    return fib(n - 1) + fib(n - 2);
           ^
main_constexpr.cpp:19:35: note: in call to 'fib(10)'
    constexpr std::uint64_t res = fib(10);
                                  ^
1 error generated.

real    0m4,276s
user    0m4,215s
sys     0m0,041s
```

Отказ от компиляции через 4 секунды с ошибкой: слишком много времени проведено в constexpr-вычислениях. Но clang-у можно передать опцию: компилируй подольше, не бойся:

```bash
$ time clang++-8 main_constexpr.cpp  -fconstexpr-steps=900000000

real    5m38,889s
user    5m38,815s
sys     0m0,042s
```

Сделаем те же вычисления в runtime:

```c++
#include <iostream>

constexpr std::uint64_t fib(std::uint64_t n)
{
    if (n <= 1)
        return n;

    for (std::uint64_t i = 1; i < 1000; ++i)
        for (std::uint64_t j = 1; j < 1000; ++j)
            if (i * i * i + j * j * j == n * n * n)
                return 0;

    return fib(n - 1) + fib(n - 2);
}

int main(int argc, char** argv)
{
    volatile std::uint64_t n = 10;
    std::cout << fib(n) << std::endl;
    return 0;
}
```

что скажет gcc?

```bash
$ g++ main_constexpr_runtime.cpp && time ./a.out
55

real    0m0,265s
user    0m0,261s
sys     0m0,004
```

clang?

```bash
$ clang++-8 main_constexpr_runtime.cpp && time ./a.out
55

real    0m0,274s
user    0m0,270s
sys     0m0,004s
```

**Итого**:

|       ~     | gcc       |   clang   |
|:------------|:---------:|:---------:|
|compiletime  |   19 sec  |   339 sec |
|runtime      |   0.3 sec |   0.3 sec |

Результаты будут зависеть от железки и алгоритма, но в целом стоит иметь ввиду, что compiletime пока что на 2 порядка медленее, чем runtime.

В рамках WG комитета стандартизации есть обсуждения как исправить такой разрыв, но принципиально сложно уйти от интерпретации, т.к. компилятор обязан ловить UB в compile-time вычислениях.

<br />

##### как тестировать constexpr-функции

Можно как и обычные функции. Но есть ещё вариант: `static_assert`-ами

```c++
constexpr int fib(int n)
{
    if (n <= 1)
        return n;
    return fib(n - 1) + fib(n - 2);
}

// boundary cases
static_assert(fib(0) == 0);
static_assert(fib(1) == 1);

// happy path
static_assert(fib(10) == 55);
```

**Замечания**:
* такие `static_assert`-тесты категорически не рекомендуется помещать в хедеры, иначе компилятор будет гонять эти тесты на каждое включение хедера
* прелесть в том, что если копиляция проходит, значит, проходят и тесты
* можно писать тесты прямо под кодом реализации (как это делается в других языках), и это не даст оверхеда на бинарный файл

<br />

##### constexpr in practice

**Пример (hardcore)**: Порверка строк на регулярное выражение.

* Выражение известно во время компиляции.
* Проверяемые строки не известны
* Компилятор на этапе компиляции генерирует конечный автомат, проверяющий регулярки. Т.к. КА известен на этапе компиляции, его можно заоптимизировать до безобразия.

Доклад Hana Dusikova на cpprussia 2019 (дать ссылку, когда будет доступен доклад).
Пока что показать [слайды с результатами](https://compile-time.re/cpprussia-piter/slides/#/13/2/6).

Исходные коды:
https://github.com/hanickadot/compile-time-regular-expressions (показать README)


Вариант использования: проверка, что строка - корректный email. RE на email известен во время компиляции.

<br />

**Пример (hardcore)**: проверка текста на соответствие грамматике (например, LL(1)) - грамматика вывода известна на этапе компиляции.

Вариант использования: парсеры для языков программирования как часть компиляторов.

<br />

**Пример:** user literals (since C++11)

```c++
constexpr double operator"" _deg(double deg)
{
    return deg * 3.14159265358979323846264L / 180;
}

constexpr double operator"" _mm(double value)
{
    return value / 1000.0;
}

constexpr double operator"" _cm(double value)
{
    return value / 100.0;
}

constexpr double operator"" _inch(double value)
{
    return value * 0.0254;
}

// usage:
constexpr auto min_sphere_size = 2_mm;
constexpr auto angle_step  = 3_deg;
```

<br />

**Пример**: проверки программиста (пока нет C++23 и compile-time интроспекции)

```c++
// DebugFlags.h
enum class DebugFlag
{
    VisualizeAlgorithm_1 = 0,
    VisualizeAlgorithm_2,
    VisualizeAlgorithm_3,
    VisualizeAlgorithm_4
};

bool isDebugFlagEnabled(DebugFlag flag);


// DebugFlags.cpp

const std::array<std::string_view, 4> registry_names = {
    "VisualizeAlgorithm_1",
    "VisualizeAlgorithm_2",
    "VisualizeAlgorithm_3",
    "VisualizeAlgorithm_4"
};

bool isDebugFlagEnabled(DebugFlag flag) {
    return getRegistryValue(registry_names[flag]) != 0;
}
```

*Замечания*:
1. очень не хочется вытаскивать `registry_names` в хедер, т.к. это, по сути, внутренняя информация, и внешнему коду про неё знать не надо, и компилировать не надо.
2. но надо как-то гарантировать, что кол-во флагов совпадает с размером `registry_names`. Сейчас мы можем посчитать руками, но вдруг кто-то умный поменяет enum, а про имена забудет?

Вариант (далёк от идеала, но интроспекция пока в процессе стандартизации):

```c++
// DebugFlags.h
enum class DebugFlag
{
    VisualizeAlgorithm_1 = 0,
    VisualizeAlgorithm_2,
    VisualizeAlgorithm_3,
    VisualizeAlgorithm_4,
    FlagsCount  // <--------- NOTE: always last, it is common in C++
};

bool isDebugFlagEnabled(DebugFlag flag);


// DebugFlags.cpp

constexpr std::array<std::string_view, 4> registry_names = {
    "VisualizeAlgorithm_1",
    "VisualizeAlgorithm_2",
    "VisualizeAlgorithm_3",
    "VisualizeAlgorithm_4"
};
// проверим, что нужное число имён есть
static_assert(static_cast<int>(DebugFlag::FlagsCount) == 4,
              "Did you forget to add registry name to debug flag?");

// проверим, что имена уникальны
consteval bool areAllNamesUnique()
{
    for (int i = 0; i < static_cast<int>(DebugFlag::FlagsCount); ++i)
        if (std::count(registry_names.begin(), registry_names.end(), registry_names[i]) != 1)
            return false;
    return true;
}
static_assert(areAllNamesUnique(),
              "Did you make just a copy-paste of registry name? Names must be unique");

bool isDebugFlagEnabled(DebugFlag flag) {
    return getRegistryValue(registry_names[flag]) != 0;
}
```

<br />

**Резюме:**
* compile-time вычисления позволяют убрать вычисления из runtime, но:
    * все входные данные должны быть известны на этапе компиляции
    * код, который позволено выполнять в compile-time, ограничен по возможностям
* До С++11 весь compile-time был в метапрограммировании, с С++11 он начинает развиваться.
* У вычислений есть 2 контекста: runtime && compiletime
    * обычная функция - runtime only
    * `constexpr`-функция - runtime && compiletime (зависит от вызова) (since C++11)
    * `consteval`-функция - compiletime only (since C++20)
* Класс допустимых `constexpr` вычислений растёт с развитием стандарта, и ещё есть к чему стремиться
* `constexpr`-вычисления медленные, т.к. компилятор интерпретирует код
* `constexpr`-вычисления позволяют отлавливать UB (since C++11) и утечки (since C++20). Если компилятор в процессе интерпретации встречает UB, он завершается с ошибкой.
    * Надо писать compile-time тесты (`static_assert` в помощь)
* `if constexpr` позволяет упростить шаблонный код (since C++17)
* `constinit` - для compile-time инициализации изменяемых переменных (since C++20)
* В С++23 обещают compile-time интроспекцию кода.